<a href="https://colab.research.google.com/github/rutuja-patil24/CMPE-258-Deep_Learning/blob/main/Assignment_05/03_Advance_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ Install required libraries
!pip install --quiet tensorflow datasets transformers scikit-learn

import tensorflow as tf
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import TFAutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-

In [2]:
# ✅ Load IMDb Dataset
dataset = load_dataset("imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
# ✅ Convert dataset to Pandas DataFrame
df = pd.DataFrame(dataset['train'])
df = df[['text', 'label']]
df['label'] = df['label'].astype(int)

In [4]:
# ✅ Adding a Second Label: Review Length Category
df['length_category'] = df['text'].apply(lambda x: 0 if len(x) < 100 else (1 if len(x) < 300 else 2))

In [5]:
# ✅ Load Tokenizer
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
# ✅ Tokenization Function
def tokenize_data(texts):
    return tokenizer(list(texts), padding="max_length", truncation=True, max_length=128, return_tensors="tf")

In [7]:
# ✅ Split Dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].values, df[['label', 'length_category']].values, test_size=0.2, random_state=42
)

train_encodings = tokenize_data(train_texts)
val_encodings = tokenize_data(val_texts)

In [8]:
# ✅ Convert labels into NumPy arrays
train_sentiment_labels = np.array([label[0] for label in train_labels])  # Sentiment (0/1)
train_length_labels = np.array([label[1] for label in train_labels])  # Length Category (0/1/2)

val_sentiment_labels = np.array([label[0] for label in val_labels])
val_length_labels = np.array([label[1] for label in val_labels])

In [9]:
# ✅ Reduce Batch Size to Prevent OOM
BATCH_SIZE = 8

In [10]:
# ✅ Convert dataset into TensorFlow format
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"]},
    {"sentiment_output": train_sentiment_labels, "length_output": train_length_labels}
)).shuffle(len(train_texts)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"]},
    {"sentiment_output": val_sentiment_labels, "length_output": val_length_labels}
)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [11]:
# ✅ Load RoBERTa Transformer Model (without classification head)
roberta_model = TFAutoModel.from_pretrained("roberta-base", return_dict=True)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.embeddings.position_ids', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [12]:
# ✅ Define the Custom Keras Model
def build_advanced_model():
    input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask")

    # ✅ Pass inputs to RoBERTa
    roberta_outputs = roberta_model(input_ids=input_ids, attention_mask=attention_mask)

    # ✅ Extract the CLS token embedding
    cls_token = tf.keras.layers.Lambda(lambda x: x.last_hidden_state[:, 0, :])(roberta_outputs)

    # ✅ Fully Connected Layers for Multi-Label Classification
    dense = tf.keras.layers.Dense(128, activation="relu")(cls_token)
    dropout = tf.keras.layers.Dropout(0.3)(dense)

    # 🔹 Two Outputs (Sentiment Prediction + Review Length Prediction)
    sentiment_output = tf.keras.layers.Dense(1, activation="sigmoid", name="sentiment_output")(dropout)
    length_output = tf.keras.layers.Dense(3, activation="softmax", name="length_output")(dropout)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=[sentiment_output, length_output])
    return model

In [13]:
# ✅ Compile Model
multi_label_model = build_advanced_model()
multi_label_model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=2e-5, weight_decay=0.01),
    loss={"sentiment_output": "binary_crossentropy", "length_output": "sparse_categorical_crossentropy"},
    metrics={"sentiment_output": "accuracy", "length_output": "accuracy"}
)

In [14]:
# ✅ Train Model
history = multi_label_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1  # Reduce epochs for quick training
)


2500/2500 [==============================] - 729s 274ms/step - loss: 0.3970 - sentiment_output_loss: 0.3444 - length_output_loss: 0.0526 - sentiment_output_accuracy: 0.8519 - length_output_accuracy: 0.9840 - val_loss: 0.3231 - val_sentiment_output_loss: 0.2791 - val_length_output_loss: 0.0440 - val_sentiment_output_accuracy: 0.8880 - val_length_output_accuracy: 0.9890


In [15]:
# ✅ Evaluate Model
results = multi_label_model.evaluate(val_dataset)
test_loss, sentiment_loss, length_loss, sentiment_acc, length_acc = results

print(f"\n✅ Test Loss: {test_loss:.4f}")
print(f"✅ Sentiment Accuracy: {sentiment_acc:.4f} | Review Length Accuracy: {length_acc:.4f}")

625/625 [==============================] - 52s 82ms/step - loss: 0.3231 - sentiment_output_loss: 0.2791 - length_output_loss: 0.0440 - sentiment_output_accuracy: 0.8880 - length_output_accuracy: 0.9890

✅ Test Loss: 0.3231
✅ Sentiment Accuracy: 0.8880 | Review Length Accuracy: 0.9890


In [20]:
# ✅ Function to Predict and Visualize
def predict_movie_review(text_samples):
    encodings = tokenizer(text_samples, padding="max_length", truncation=True, max_length=128, return_tensors="tf")

    inputs = {
        "input_ids": encodings["input_ids"],
        "attention_mask": encodings["attention_mask"]
    }

    # ✅ Make Predictions
    predictions = multi_label_model.predict(inputs)

    sentiment_preds = predictions[0].flatten()
    length_preds = np.argmax(predictions[1], axis=1)

    for text, sentiment, length in zip(text_samples, sentiment_preds, length_preds):
        sentiment_label = "Positive" if sentiment > 0.5 else "Negative"
        length_label = ["Short", "Medium", "Long"][length]
        print(f"🎬 Review: {text[:100]}... → **Sentiment: {sentiment_label}, Length: {length_label}**")


In [21]:
# ✅ Example Predictions
sample_reviews = [
    "Absolutely loved this movie! The cinematography and acting were top-notch!",
    "Worst movie ever. Poor script and terrible acting.",
    "It was okay, not the best but watchable."
]

predict_movie_review(sample_reviews)


1/1 [==============================] - 6s 6s/step
🎬 Review: Absolutely loved this movie! The cinematography and acting were top-notch!... → **Sentiment: Negative, Length: Medium**
🎬 Review: Worst movie ever. Poor script and terrible acting.... → **Sentiment: Negative, Length: Medium**
🎬 Review: It was okay, not the best but watchable.... → **Sentiment: Negative, Length: Medium**
